In [40]:
import pandas as pd
import hashlib
import json
import uuid
from collections import deque

In [41]:
def generate_uuid(value):
    return str(uuid.UUID(hashlib.md5(value.encode()).hexdigest()))

def allocate_questions(questions, doctors):
    allocation = {doctor: [] for doctor in doctors}
    
    for i, question in enumerate(questions):
        doctor = doctors[i % len(doctors)]  # Assign in round-robin manner
        allocation[doctor].append(question)
    
    return allocation

def rotate_list_in_place(lst):
    dq = deque(lst)
    dq.rotate(1)  # Rotate right by 1
    lst[:] = list(dq)  # Update original list in place


In [42]:
import pandas as pd

questions_df = pd.DataFrame({
    'question': [
        "Can I eat breakfast before the surgery?",
        "What are the dos and donts after surgery?",
        "How safe is the surgery?",
        "How often should eye drops be applied before surgery?",
        "Operated eye has turned reddish. What to do?"
    ],
    'answer': [
        "Before cataract surgery, you’ll usually be asked not to eat or drink anything for at least 6 hours beforehand. This is to reduce the risk of complications from anesthesia. If your surgery is in the morning, you’ll likely need to skip breakfast. However, some clinics allow small sips of water or clear fluids up to 2 hours before. Always follow the exact instructions your doctor or surgical team gives you—they’ll tell you what’s safe for your specific case. If you’re unsure, call the clinic to confirm.",
        "After cataract surgery, do rest your eyes, use prescribed eye drops, and wear the protective shield as advised. Keep the eye clean and avoid rubbing it. Don’t lift heavy objects, bend over deeply, swim, or expose your eye to dust or dirty water for at least a week. Avoid eye makeup and strenuous exercise until your doctor says it’s safe. Wear sunglasses outdoors to protect from bright light. Attend all follow-up appointments to monitor healing. If you notice pain, worsening vision, or redness, contact your doctor promptly. Always follow your surgeon’s specific instructions for the best recovery.",
        "Cataract surgery is very safe and one of the most common surgeries worldwide. Over 95% of cases improve vision without serious complications. Modern techniques use small incisions and quick recovery times. Risks like infection, swelling, or retinal detachment are rare but possible. Most side effects, such as mild discomfort or blurred vision, are temporary and resolve within days or weeks. Your surgeon will check your eye health beforehand to minimize risks. Following aftercare instructions helps ensure a smooth recovery. If you have other eye conditions, discuss them with your doctor to understand your personal risk better.",
        "Most patients need to start using antibiotic and anti-inflammatory eye drops 2 to 3 days before surgery. They are usually applied 3 to 4 times a day. Your doctor will give you exact instructions based on your eye’s needs. Be sure to follow the schedule carefully for the best results.",
        "If your operated eye has turned red, it could be normal mild irritation, but it might also be a sign of infection or pressure issues. Do not wait—contact your eye doctor right away to check it. Avoid rubbing your eye and keep using your prescribed drops unless told otherwise."
    ]
})


In [43]:
docs_df = pd.read_csv("/home/rash598/doctor_sys_3/cataract-doc-study/cataract_doc_study/data/doctor.csv")

In [44]:
docs_df['id'] = docs_df['email'].apply(generate_uuid)
questions_df['id'] = questions_df['question'].apply(generate_uuid)

In [45]:
doctor_id = "doctor_id"
condition_id = "condition_id"
question_id = "question_id"
duration = "duration"

columns = [doctor_id, condition_id, question_id, duration, "answer", "conversation"]
doctors_question_set = pd.DataFrame(columns=columns)

In [46]:
doctor_id_list = docs_df['id'].tolist()
questions_id_list = questions_df['id'].tolist()

for i in range(3):
    for doc_id in doctor_id_list:
        if i == 0:
            end = 3
        else:
            end = len(doctor_id_list)
        for q_id in questions_id_list[:end]:
            new_row = {
                doctor_id: doc_id,
                condition_id: i,
                question_id: q_id,
                duration: 0,
                "answer": "",
                "conversation": ""
            }
            doctors_question_set = pd.concat([doctors_question_set, pd.DataFrame([new_row])], ignore_index=True)

In [47]:
doctors_question_set

,doctor_id,condition_id,question_id,duration,answer,conversation
0,90a06836-7e7c-95a4-bd7b-274f44cbc0da,0,e6cf982f-c69b-d70c-c4f3-0c1f26b12fd9,0,,
1,90a06836-7e7c-95a4-bd7b-274f44cbc0da,0,cc7e0bd4-ac0f-6fbd-12ea-a26e566b8bd7,0,,
2,90a06836-7e7c-95a4-bd7b-274f44cbc0da,0,1cc63bda-98aa-96cb-23f6-a91fe19b6bc6,0,,
3,619744a7-7d59-4835-a856-e882a9d357c1,0,e6cf982f-c69b-d70c-c4f3-0c1f26b12fd9,0,,
4,619744a7-7d59-4835-a856-e882a9d357c1,0,cc7e0bd4-ac0f-6fbd-12ea-a26e566b8bd7,0,,
...,...,...,...,...,...,...
294,947b6665-d9e2-217a-2522-b5da67d87d75,2,e6cf982f-c69b-d70c-c4f3-0c1f26b12fd9,0,,
295,947b6665-d9e2-217a-2522-b5da67d87d75,2,cc7e0bd4-ac0f-6fbd-12ea-a26e566b8bd7,0,,
296,947b6665-d9e2-217a-2522-b5da67d87d75,2,1cc63bda-98aa-96cb-23f6-a91fe19b6bc6,0,,
297,947b6665-d9e2-217a-2522-b5da67d87d75,2,8311e038-832a-34f9-c8a7-3446e697c80d,0,,


In [48]:
docs_json = docs_df.to_dict(orient="records")
docs_df.to_json("pilot_survey_doctors.json", orient="records", indent=4)  # `indent=4` for readable formatting
questions_df.to_json("pilot_surevy_questions.json", orient="records", indent=4) 

In [49]:
# Grouping by doctor_id
doctors_question_set_json = []
for user_id, group in doctors_question_set.groupby("doctor_id"):
    questions_list = group[["question_id", "condition_id"]].to_dict(orient="records")
    doctors_question_set_json.append({
        "user_id": user_id,
        "questions_list": questions_list,
        "progress_id": 0
    })
doctors_question_set_json

[{'user_id': '07dccc31-15c3-4395-2e0f-d34a7b6452cf',
  'questions_list': [{'question_id': 'e6cf982f-c69b-d70c-c4f3-0c1f26b12fd9',
    'condition_id': 0},
   {'question_id': 'cc7e0bd4-ac0f-6fbd-12ea-a26e566b8bd7', 'condition_id': 0},
   {'question_id': '1cc63bda-98aa-96cb-23f6-a91fe19b6bc6', 'condition_id': 0},
   {'question_id': 'e6cf982f-c69b-d70c-c4f3-0c1f26b12fd9', 'condition_id': 1},
   {'question_id': 'cc7e0bd4-ac0f-6fbd-12ea-a26e566b8bd7', 'condition_id': 1},
   {'question_id': '1cc63bda-98aa-96cb-23f6-a91fe19b6bc6', 'condition_id': 1},
   {'question_id': '8311e038-832a-34f9-c8a7-3446e697c80d', 'condition_id': 1},
   {'question_id': '56e16d01-4f8d-988e-e480-31a9c38409e6', 'condition_id': 1},
   {'question_id': 'e6cf982f-c69b-d70c-c4f3-0c1f26b12fd9', 'condition_id': 2},
   {'question_id': 'cc7e0bd4-ac0f-6fbd-12ea-a26e566b8bd7', 'condition_id': 2},
   {'question_id': '1cc63bda-98aa-96cb-23f6-a91fe19b6bc6', 'condition_id': 2},
   {'question_id': '8311e038-832a-34f9-c8a7-3446e697c80d

In [50]:
with open("pilot_survey_doctor_question_set.json", "w") as f:
    json.dump(doctors_question_set_json, f, indent=4)

In [51]:
from cataract_doc_study.dependency_setup import user_client, survey_client

documents = []
ids = await user_client.afetch_ids()
for doctor_question_set in doctors_question_set_json:
    if doctor_question_set["user_id"] in ids:
        continue  # Skip if the user already exists
    question_set = {
        "_id": doctor_question_set["user_id"],
        "user_id": doctor_question_set["user_id"],
        "questions_list": doctor_question_set["questions_list"],
        "progress_id": doctor_question_set["progress_id"],
    }
    documents.append(question_set)
await user_client.ainsert(documents)

(['1cd5568a-435d-fa0a-9009-15b9cc196a2a',
  '1cefb786-ac3d-81c3-9ad9-1fe6580175e2',
  '26680f63-b80e-6efb-0669-ed0c200e58ef',
  '7e8e5c3b-9b52-e06a-e208-366eff9f4c6d',
  '947b6665-d9e2-217a-2522-b5da67d87d75'],
 None)